In [2]:
# Import modules

In [3]:
import os
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import operator
import json
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
import seaborn as sns
import pandas as pd
import gensim
from gensim import corpora
import pyLDAvis.gensim
import IPython.display
import pyLDAvis
import pyLDAvis.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/a.a.gonzalez.paje/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Set up working directory

In [5]:
os.chdir('/Users/a.a.gonzalez.paje/Box Sync/Alberto/rapid_intel/3.0/code')
os.getcwd()

'/Users/a.a.gonzalez.paje/Box Sync/Alberto/rapid_intel/3.0/code'

In [6]:
# Load listener
term_to_search = 'royalcaribbean'
output_file_name = 'listener_results_' + term_to_search + '.json'
print output_file_name

listener_results_royalcaribbean.json


In [7]:
# Load json with tweets
input_file = output_file_name
df = pd.read_json(input_file, orient = 'records', lines = True)
print df.shape
#print df.head(2)
#print df.columns.values

(66, 36)


In [8]:
# Cleaning
def clean_tweet(tweet):
     '''
     Utility function to clean the text in a tweet by removing
     links and special characters using regex.
     '''
     return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [9]:
# Apply the cleaning function above to the column in a dataframe with tweet texts and print head (first 5 rows)
df['text_clean'] = np.array([clean_tweet(tweet) for tweet in df['text']])
print df['text_clean'].head() 

0              Last cruise sunset enchantmentoftheseas
1             LOVED cococayisland enchantmentoftheseas
2    Cruise date at Chops Steakhouse on enchantment...
3    Vacation is over but we had a blast with frien...
4    RT SundayFunday tip run towards the pool run a...
Name: text_clean, dtype: object


In [10]:
stop_it = stopwords.words('english') + ['rt','RT','via','️','❗','…',"0","1","2","3","4","5","6","7","8","9"]

def clean_tweet_ngrams(tweet,stemming=False):
     '''
     Utility function to clean the text in a tweet by trasnforming to lower case,
     stemming and removing stopwords
     '''
     tweet = tweet.lower()
     tweet = tweet.split()
     if stemming==False:
         tweet = [word for word in tweet if not word in stop_it]
     else:
         ps = PorterStemmer()
         tweet = [ps.stem(word) for word in tweet if not word in set(stopwords.words('english'))]
     tweet = ' '.join(tweet)
     return tweet


In [11]:
#Apply the cleaning function above to the dataframe column with text to clean and print head (first 5 rows)
# can be applied to text preprocessed with clean_tweet function for further pre-processing
df['text_clean_ngrams'] = np.array([ clean_tweet_ngrams(tweet) for tweet in df['text_clean'] ])    
print df['text_clean_ngrams'].head() 

0              last cruise sunset enchantmentoftheseas
1             loved cococayisland enchantmentoftheseas
2    cruise date chops steakhouse enchantmentofthes...
3    vacation blast friends old new safe travels fr...
4    sundayfunday tip run towards pool run away monday
Name: text_clean_ngrams, dtype: object


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  # This is added back by InteractiveShellApp.init_path()


In [12]:
'''
Transforming to lowercase is optional and can be controlled via the 'lowercase' attribute: set 'lowercase' to 'True' 
to apply transformation to lower case
'''

"\nTransforming to lowercase is optional and can be controlled via the 'lowercase' attribute: set 'lowercase' to 'True' \nto apply transformation to lower case\n"

In [13]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [14]:
# Term frequencies

In [15]:
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via','️','❗','…']

def counting_terms(term_type='terms_stop', fname=input_file,nwords =5):
    with open(fname, 'r') as f:
        count_all=Counter()
        for line in f:
            tweet=json.loads(line)
            # Create a list with  terms
            if term_type == 'terms_all':
                terms_all = [term for term in preprocess(tweet['text'],True)]            
                # Update the counter
                count_all.update(terms_all)
            elif term_type == 'terms_stop':
                terms_stop = [term for term in preprocess(tweet['text'],True) if term not in stop]               
                # Update the counter
                count_all.update(terms_stop)
            elif term_type == 'terms_hash':
                terms_hash = [term for term in preprocess(tweet['text'],True) 
                  if term.startswith('#')]               
                # Update the counter
                count_all.update(terms_hash)
            else:
                terms_only = [term for term in preprocess(tweet['text'],True) 
                  if term not in stop and
                  not term.startswith(('#', '@'))] 
                count_all.update(terms_only)
        # Print the first 5 most frequent words
        print(count_all.most_common(nwords))
        return count_all

In [16]:
# All terms
term_freq=counting_terms(term_type='terms_all', fname=input_file, nwords = 10)

[(u'.', 46), (u'@royalcaribbean', 43), (u'\u2026', 38), (u':', 35), (u',', 29), (u'the', 27), (u'for', 20), (u'rt', 18), (u'you', 18), (u'cruise', 15)]


In [17]:
# Excluding stopwords
term_freq_stop=counting_terms(term_type='terms_stop', fname=input_file,nwords=10)

[(u'@royalcaribbean', 43), (u'\u2026', 38), (u'cruise', 15), (u'babysitting', 12), (u'\u2019', 11), (u'in-room', 11), (u'sea', 10), (u'royal', 10), (u'caribbean', 10), (u'sitter', 9)]


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  from ipykernel import kernelapp as app


In [18]:
# Only hashtags
term_freq_hash=counting_terms(term_type='terms_hash', fname=input_file,nwords=10)

[(u'#royalcaribbean', 7), (u'#navigatoroftheseas', 4), (u'#enchantmentoftheseas', 3), (u'#kauai', 2), (u'#sundayfunday', 2), (u'#radianceoftheseas', 2), (u'#cruise', 2), (u'#honduras', 1), (u'#caribbean', 1), (u'#cococayisland', 1)]


In [19]:
# Terms only
term_freq_only=counting_terms(term_type='terms_only', fname=input_file,nwords=10)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:25: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


[(u'\u2026', 38), (u'cruise', 15), (u'babysitting', 12), (u'\u2019', 11), (u'in-room', 11), (u'sea', 10), (u'royal', 10), (u'caribbean', 10), (u'sitter', 9), (u'program', 8)]


In [20]:
# Bigrams

In [21]:
tokens = df['text_clean_ngrams'].apply(nltk.word_tokenize) 
# Flatening nested list
flat_tokens = [term for sublist in tokens for term in sublist]

bgs = nltk.bigrams(flat_tokens)

In [22]:
fdist = nltk.FreqDist(bgs)
#for k,v in fdist.items():
    #print(k,v)

fdist_10 = fdist.most_common(10)

In [23]:
print fdist_10

[((u'room', u'babysitting'), 12), ((u'royal', u'caribbean'), 10), ((u'sitter', u'sea'), 9), ((u'terminates', u'room'), 8), ((u'popular', u'cruise'), 8), ((u'caribbean', u'discontinues'), 8), ((u'sea', u'program'), 8), ((u'line', u'terminates'), 8), ((u'babysitting', u'service'), 8), ((u'cruise', u'line'), 8)]


In [24]:
labels = ['bigram', 'Weight']

df_bigrams = pd.DataFrame([tuple_item for tuple_item in fdist.items()], columns =labels)

df_bigrams[['Source_Name','Target_Name']]=pd.DataFrame([tuple_item for tuple_item in df_bigrams.bigram])

#sort in descending order
df_bigrams.sort_values(by='Weight',ascending=False).head(3)

,bigram,Weight,Source_Name,Target_Name
303,"(room, babysitting)",12,room,babysitting
104,"(royal, caribbean)",10,royal,caribbean
240,"(sitter, sea)",9,sitter,sea


In [25]:
df_bigrams_sub=df_bigrams[df_bigrams['Weight']>0] # Change your weighting filter as needed
print df_bigrams_sub.head(3)

                bigram  Weight Source_Name Target_Name
0     (known, diverse)       1       known     diverse
1         (next, week)       1        next        week
2  (quantum, brochure)       1     quantum    brochure


In [26]:
# Reshaping format to create output tables
#Select multi-level index columns
idx =['bigram','Weight']

# Then pivot the dataset based on this multi-level index 
multi_indexed_df = df_bigrams_sub.set_index(idx)
multi_indexed_df.head(2)
multi_indexed_df.columns.name = 'word1_2'

In [27]:
# Stack the columns to achieve the baseline long format for the data
stacked_df = multi_indexed_df.stack(dropna=False)
stacked_df.name = 'Label'
stacked_df.head()

bigram               Weight  word1_2    
(known, diverse)     1       Source_Name      known
                             Target_Name    diverse
(next, week)         1       Source_Name       next
                             Target_Name       week
(quantum, brochure)  1       Source_Name    quantum
Name: Label, dtype: object

In [28]:
# Reset index
stacked_df=stacked_df.reset_index()
print stacked_df.head(3)

             bigram  Weight      word1_2    Label
0  (known, diverse)       1  Source_Name    known
1  (known, diverse)       1  Target_Name  diverse
2      (next, week)       1  Source_Name     next


In [29]:
# Nodes Table
# Generate list with unique bigrams
print('shape stacked_df:',stacked_df.shape)
Label_unique=stacked_df.Label.unique()
print Label_unique[0:2]
print 'length unique labels:',len(Label_unique)

('shape stacked_df:', (778, 4))
[u'known' u'diverse']
length unique labels: 308


In [30]:
gephi1 = []
i=0
for item in Label_unique: 
    i=i+1
    gephi1.append({'Label': item, 'Id': i})

gephi1=pd.DataFrame(gephi1)
print gephi1.head(3)

   Id    Label
0   1    known
1   2  diverse
2   3     next


In [31]:
mydict={}
i = 0
for item in Label_unique:    
    i = i+1
    mydict[item] = i

In [32]:
output_file_name_nodes = 'bigrams_nodes_table_' + term_to_search + '.csv'
print output_file_name_nodes

bigrams_nodes_table_royalcaribbean.csv


In [33]:
gephi1[['Id','Label']].to_csv(output_file_name_nodes,index=False)
print(gephi1.head(3))

   Id    Label
0   1    known
1   2  diverse
2   3     next


In [34]:
# Relations Table
df_bigrams_sub['Source']=np.array([mydict[item] for item in df_bigrams_sub['Source_Name']])
df_bigrams_sub['Target']=np.array([mydict[item] for item in df_bigrams_sub['Target_Name']])
print df_bigrams_sub.head(2)

             bigram  Weight Source_Name Target_Name  Source  Target
0  (known, diverse)       1       known     diverse       1       2
1      (next, week)       1        next        week       3       4


In [35]:
output_file_name_relations = 'bigrams_relations_table_' + term_to_search + '.csv'
print output_file_name_relations

bigrams_relations_table_royalcaribbean.csv


In [36]:
# In this case, the relations table is generated using nodes lables, not nodes ids.
df_bigrams_sub[['Source','Target','Weight']].to_csv(output_file_name_relations,index=False)

In [ ]:
# Sentiment Analysis

In [38]:
def analize_sentiment(tweet):
    '''
    Utility function to classify the polarity of a tweet
    using textblob    '''
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1

In [39]:
df['SA'] = np.array([analize_sentiment(tweet) for tweet in df['text_clean']])

pos_tweets = [tweet for index, tweet in enumerate(df['text']) if df['SA'][index] > 0]
print "Percentage of positive tweets: {}%".format(len(pos_tweets)*100/len(df['text']))

neu_tweets = [tweet for index, tweet in enumerate(df['text']) if df['SA'][index] == 0]
print "Percentage of neutral tweets: {}%".format(len(neu_tweets)*100/len(df['text']))

neg_tweets = [tweet for index, tweet in enumerate(df['text']) if df['SA'][index] < 0]
print "Percentage of negative tweets: {}%".format(len(neg_tweets)*100/len(df['text']))

Percentage of positive tweets: 45%
Percentage of neutral tweets: 42%
Percentage of negative tweets: 12%


In [40]:
# Creating tables with positive, neutral and negative results
neg_tweets_df = df[df['SA']<0]
neu_tweets_df = df[df['SA']==0]
pos_tweets_df = df[df['SA']>0]

In [42]:
# Print first negative texts
print neg_tweets_df['text_clean'].head(10) 

9     RT Kauai you didn t disappoint in dramatic sce...
13        No more in room babysitting on royalcaribbean
16    RT Kauai you didn t disappoint in dramatic sce...
27    very poor experience on Quantum not at all wha...
50    Booked my Nov cruise back in Feb Price of balc...
51    It s very disappointing when you have to cance...
61    Random pictures from the cruise ohshipgirlstri...
62    RT Big boat small alley big theatre small scot...
Name: text_clean, dtype: object


In [ ]:
# Topic Modeling

In [43]:
# Create the dictionary where each term is assigned an index
dictionary = corpora.Dictionary(tokens)
print dictionary

Dictionary(308 unique tokens: [u'roger', u'approx', u'since', u'respectthewate', u'help']...)


In [46]:
doc_term_matrix = [dictionary.doc2bow(tweet) for tweet in tokens]

In [56]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50) # Change the number of topics as needed

In [57]:
lda_vis = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)

/anaconda2/lib/python2.7/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [58]:
# The following code will generate an HTML file with the Topic Modelling results
pyLDAvis.enable_notebook(local=False)
pyLDAvis.save_html(lda_vis,"data/topic_modeling_twitter.html") # Change the output name as needed
pyLDAvis.display(lda_vis)

In [59]:
pyLDAvis.enable_notebook()

In [60]:
# TF-IDF
tf_vectorizer = CountVectorizer(min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(df['text_clean_ngrams'])
tf_feature_names = tf_vectorizer.get_feature_names()
print dtm_tf.shape

(66, 6)


In [61]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(df['text_clean_ngrams'])
print dtm_tfidf.shape

(66, 6)


In [62]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=5, random_state=0)
lda_tf.fit(dtm_tf)

/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=5, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [63]:
def display_topics(model, feature_names, no_top_words): 
    for topic_idx, topic in enumerate(model.components_): 
        print("Topic %d:" % (topic_idx)) 
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [64]:
no_top_words = 10
display_topics(lda_tf, tf_feature_names, no_top_words)

Topic 0:
cruise babysitting sea caribbean royal room
Topic 1:
babysitting room sea royal caribbean cruise
Topic 2:
cruise royal caribbean sea room babysitting
Topic 3:
room babysitting cruise royal sea caribbean
Topic 4:
sea room babysitting caribbean royal cruise
